In [1]:
#from __future__ import absolute_import
#from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.advanced_activations import PReLU
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, Adadelta, Adagrad
from keras.utils import np_utils, generic_utils
from keras.layers import Input
from six.moves import range
import cPickle as pickle
import cv2

import os
from PIL import Image
import numpy as np

Using Theano backend.
/usr/local/lib/python2.7/dist-packages/theano/misc/pycuda_init.py:35: UserWarning: PyCUDA import failed in theano.misc.pycuda_init
  warnings.warn("PyCUDA import failed in theano.misc.pycuda_init")
Using gpu device 0: GeForce GT 740M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5105)


In [ ]:
# Read folder mnist under 42,000 pictures, pictures to grayscale, so as a channel,
# If a color map as input, then a replacement is 3, and the data [i,:,:,:] = arr to Data [i,:,:,:] = [arr [:,:, 0], arr [:,:, 1], arr [:,:, 2]]
def  load_data():
    imgs = os.listdir("./train")
    num = len(imgs)
    data = np.empty((num,3,32,32),dtype="float32")
    label = np.empty((num,1),dtype ="uint8")
    j=0
    for i in range(num):
        if not imgs[i].startswith('.'):
            label[j]=0 if imgs[i].split('.')[0]=='cat' else 1
            img = Image.open("./train/"+imgs[i])
            arr = np.asarray (img, dtype ="float32")
            if len(arr.shape) == 2:
                tmp = np.zeros((32, 32), dtype="float32")
                arr = [arr, tmp, tmp]
                arr = np.reshape(arr, (32,32,3))
            print j
            print imgs[i].split('.')[0]+str(label[j])
            data [j,:,:,:] = [arr[:,:,0],arr[:,:, 1],arr[:,:, 2]]
            j=j+1
    return data, label



# # Load data
data,label = load_data()

##label 0 to 9 of 10 categories, keras requested format is binary class matrices, transforming it, directly call this function keras provided
# label = np_utils.to_categorical(label,2)

0
cat[0]
1
dog[1]
2
dog[1]
3
dog[1]
4
dog[1]
5
dog[1]
6
dog[1]
7
dog[1]
8
dog[1]
9
dog[1]
10
dog[1]
11
dog[1]
12
dog[1]
13
dog[1]
14
dog[1]
15
dog[1]
16
dog[1]
17
dog[1]
18
dog[1]
19
cat[0]
20
cat[0]
21
dog[1]
22
dog[1]
23
cat[0]
24
dog[1]
25
cat[0]
26
dog[1]
27
dog[1]
28
dog[1]
29
dog[1]
30
dog[1]
31
dog[1]
32
cat[0]
33
cat[0]
34
dog[1]
35
dog[1]
36
dog[1]
37
dog[1]
38
dog[1]
39
dog[1]
40
cat[0]
41
dog[1]
42
dog[1]
43
dog[1]
44
dog[1]
45
dog[1]
46
dog[1]
47
dog[1]
48
dog[1]
49
dog[1]
50
dog[1]
51
cat[0]
52
dog[1]
53
dog[1]
54
dog[1]
55
cat[0]
56
dog[1]
57
dog[1]
58
dog[1]
59
dog[1]
60
dog[1]
61
dog[1]
62
dog[1]
63
dog[1]
64
dog[1]
65
dog[1]
66
cat[0]
67
dog[1]
68
dog[1]
69
dog[1]
70
dog[1]
71
cat[0]
72
cat[0]
73
cat[0]
74
dog[1]
75
dog[1]
76
cat[0]
77
cat[0]
78
dog[1]
79
cat[0]
80
dog[1]
81
dog[1]
82
dog[1]
83
cat[0]
84
dog[1]
85
dog[1]
86
dog[1]
87
cat[0]
88
cat[0]
89
dog[1]
90
cat[0]
91
dog[1]
92
dog[1]
93
dog[1]
94
dog[1]
95
dog[1]
96
dog[1]
97
dog[1]
98
dog[1]
99
cat[0]
100
dog[1]

In [2]:
train_file = open('train.pkl', 'rb')
label_file = open('labels.pkl', 'rb')

# pickle.dump(data, train_file)
# pickle.dump(label, label_file)

data = pickle.load(train_file)
label = pickle.load(label_file)

train_file.close()
label_file.close()

print(data.shape[0],'samples')

(49999, 'samples')


In [29]:
print label.shape
print label
batch_size = 32
nb_epoch = 100
#data_augmentation = True

(49999, 2)
[[ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 ..., 
 [ 0.  1.]
 [ 1.  0.]
 [ 0.  1.]]


In [5]:
print data.shape
print 3*32*32

tmp = data
tmp = tmp.reshape((49999, 32, 32, 3))

(49999, 3, 32, 32)
3072


In [6]:
# model = Sequential()

# model.add(ZeroPadding2D((1,1),input_shape=(3,32,32)))
# model.add(Convolution2D(64, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(64, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(128, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(128, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(256, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(ZeroPadding2D((1,1)))
# model.add(Convolution2D(512, 3, 3, activation='relu'))
# model.add(MaxPooling2D((2,2), strides=(2,2)))

# model.add(Flatten())
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4096, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='softmax'))

In [15]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(3, 32, 32)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

epochs = 50
learning_rate = 0.1
decay_rate = learning_rate / epochs
momentum = 0.8
sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
model.fit(data, label, nb_epoch=epochs)  # starts training

Epoch 1/50
49999/49999 [==============================] - 23s - loss: 4.0857 - acc: 0.7451    
Epoch 2/50
49999/49999 [==============================] - 23s - loss: 4.0861 - acc: 0.7451    
Epoch 3/50
49999/49999 [==============================] - 23s - loss: 4.0852 - acc: 0.7452    
Epoch 4/50
49999/49999 [==============================] - 23s - loss: 4.0793 - acc: 0.7456    
Epoch 5/50
49999/49999 [==============================] - 24s - loss: 4.0872 - acc: 0.7451    
Epoch 6/50
49999/49999 [==============================] - 24s - loss: 4.0811 - acc: 0.7455    
Epoch 7/50
49999/49999 [==============================] - 23s - loss: 4.0839 - acc: 0.7453    
Epoch 8/50
49999/49999 [==============================] - 24s - loss: 4.0774 - acc: 0.7457    
Epoch 9/50
49999/49999 [==============================] - 23s - loss: 4.0772 - acc: 0.7457    
Epoch 10/50
49999/49999 [==============================] - 23s - loss: 4.0829 - acc: 0.7453    
Epoch 11/50
49999/49999 [========================

In [10]:
# sgd = SGD(lr=.5)
# model.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

# model.fit(data, label, nb_epoch=nb_epoch)  # starts training

In [31]:
img = Image.open("/home/abdulaziz/Downloads/datasets/train_2/dog.98.jpg")
arr = np.asarray (img, dtype ="float32")

print arr.shape
# arr = arr.reshape((1, 3, 32, 32))

print model.predict( arr, batch_size=32, verbose=0)

(480, 500, 3)


Exception: Error when checking : expected convolution2d_input_4 to have 4 dimensions, but got array with shape (480, 500, 3)